In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!git clone https://github.com/uonat/SS2023_DI-Lab_Precitaste.git

In [ ]:
%cd SS2023_DI-Lab_Precitaste

In [ ]:
!pip install . &> /dev/null

In [ ]:
import distutils.core
import sys,os
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'  &> /dev/null
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])} &> /dev/null
sys.path.insert(0, os.path.abspath('./detectron2'))

# Data Loading

In [ ]:
from dataset.SKU110K_fixed_dataset import download_dataset,get_image
#TODO or if already downloaded, use directly to the directory path
#path_to_ds,annotations_path = download_dataset() 

import tarfile
with tarfile.open('/content/drive/MyDrive/ApplicationProject/Data/SKU110K_fixed.tar.gz') as compressed_folder: 
    compressed_folder.extractall('/content/SS2023_DI-Lab_Precitaste/dataset')
path_to_ds = "/content/SS2023_DI-Lab_Precitaste/dataset/SKU110K_fixed"
annotations_path = "{}/annotations".format(path_to_ds)


#TODO get annotations here:  (& update print_img_with_GT_BB)
#all_annotations = {"train":_get_annotations(train_ann_path),"test":_get_annotations(test_ann_path),"val":_get_annotations(val_ann_path)} 
#print_img_with_GT_BB(img_path,annotations_path,which_set=which_set)  

# Model

In [ ]:
from models.VitDet import load_model,Draw_pred_BB,Draw_original_seg

model_path ="/content/drive/MyDrive/ApplicationProject/Models/model2.pkl"
model = load_model(model_path) 
#"/content/SS2023_DI-Lab_Precitaste/detectron2/projects/ViTDet/configs/LVIS/mask_rcnn_vitdet_h_100ep.py"

# Output

In [ ]:
import torch

which_set = "train" #train #test #val
random_img_num = 4
img_path="{}/images/{}_{}.jpg".format(path_to_ds,which_set,random_img_num) #Check if exists

img = get_image(img_path)

batch = [{'image':torch.from_numpy(img).movedim(-1,0)}]

model.to('cpu')
model.eval()
with torch.no_grad():
  model_result=model(batch)


Draw_pred_BB(img,model_result)
#Draw_original_seg(img,model_result)
